In [1]:
from odps import ODPS
from odps.models import Schema, Column, Partition

# 拥有metaapp0权限的access id和key
# o = ODPS('LTAI4Ff16YHnNBW3D6TqcTMA',
#          'Qp9djQ7vPWXtbctXvMqmACQH4qdcq2',
#          project='metaapp0',
#          endpoint='http://service.cn-beijing.maxcompute.aliyun.com/api')
o = ODPS('LTAI4G54bHC17oD3WNvrrgiL',
         'HIo65feoTnB7RyEg6IkDO90JUl1XHy',
         project='metaapp_jarvis',
         endpoint='http://service.cn-beijing.maxcompute.aliyun.com/api')


In [2]:
# 获取高频query，，可以不运行，，在训练集里再去跑所有相关query
#instance = o.execute_sql("select * from metaapp_jarvis.ljq__query_game_show_click_download_playtime__noexp where ymd='20210531_20210531' limit 100")
instance = o.execute_sql("SELECT query,cnt FROM metaapp_jarvis.ljq__log_event_search_result WHERE ymd = '20210522'")
queries = []
with instance.open_reader() as reader:
    for record in reader:  
        queries.append(record.values[0])

In [3]:
# 从redis中获取这些高频query的向量
import redis
import time

query_vectors = {}

r = redis.Redis(host='r-2zeec2xz0vtvs88fy9pd.redis.rds.aliyuncs.com', port=6379, password='UE52W79rxseB')

for _ind,query in enumerate(queries):
    time.sleep(0.001)
    if _ind%400==0:
        print(_ind)
    query_vector = r.get("QueryVector_"+query)
    if query_vector is not None: 
        query_vectors[query] = query_vector.decode()

r.close()

In [9]:
# 很多没获取到，再从预估服务中获取redis里没有的
import requests
import json 
import time

url_path = 'http://1884010027272209.cn-beijing.pai-eas.aliyuncs.com/api/predict/phrase_match0518'
headers = {
    'Authorization': 'NGUxMTA4NzMwNWEwNTlkMGUwOWNlYzFkMzk5YTc4NTllYTBjZGM5Yg=='
}

def getVectorFromEas(query):
    post_data = json.dumps({"terms":query})
    resp = requests.post(url_path, post_data, headers=headers)
    if resp is not None and resp.status_code==200:
        return json.loads(resp.content.decode())['result']['predictions']
    else:
        return ''
    

In [ ]:
# 获取不在redis里的高频query的向量
query_with_vector = query_vectors.keys()

for _ind,query in enumerate(queries):
    if _ind%400==0:
        print(_ind)
    if query not in query_with_vector:
        time.sleep(0.01)
        tmp = getVectorFromEas(query)
        if tmp!='':
            query_vectors[query] = tmp
            

In [13]:
# 将query_vectors存到本地
f = open('query_vectors','w',encoding='utf-8')
for query,vector in query_vectors.items():
    f.write(query+'\t'+vector+'\n')
f.close()

In [2]:
# 将query_vector恢复到内存
query_vectors = {}

f = open('query_vectors','r',encoding='utf-8')
for line in f.readlines():
    line = line[:-1]
    query,vector = line.split('\t')
    query_vectors[query] = vector
f.close()

In [3]:
instance = o.execute_sql("SELECT  query,gameid,if(playtime>0,playtime,0) as playtime FROM  "
                         "( "
                         "SELECT  query,gameid,COUNT(1) AS cnt,sum(playtime) AS playtime,rand() AS rd FROM  metaapp_jarvis.ljq__query_game_show_click_download_playtime__noexp "
                         "WHERE   ymd = '20210531_20210531' GROUP BY query,gameid"
                         ") "
                         "WHERE   playtime > 0 OR rd < 0.01")
train_data_main = []
with instance.open_reader() as reader:
    for record in reader:  
        train_data_main.append(record.values)
        

In [34]:
# 将原始训练数据存到本地
f = open('train_data_main','w',encoding='utf-8')
for itm in train_data_main:
    f.write(str(itm[0])+'\t'+str(itm[1])+'\t'+str(itm[2])+'\n')
f.close()

In [4]:
# 先找出训练集中出现的gameid，无关的就不放入map，省内存
gameid_set = set()
for itm in train_data_main:
    gameid_set.add(itm[1])

In [ ]:
# 通过odpscmd下载数据，用odpssql数据太大下不下来，，odpscmd下载原表由于vector是array<float>也下不下来，需转换为string再下载
# d:/odpscmd_public/bin/odpscmd -e "tunnel download -threads 6 metaapp_jarvis_dev.ljq__gameid_title_bert_vector__str/ymd=20210531 d:/table_data/ljq__gameid_title_bert_vector__str/20210531 -fd '\t'"

In [7]:
#将涉及的game的vector载入内存
gameid_vector = {}
gameid_vector_path = r'D:\table_data\ljq__gameid_title_bert_vector__str\20210531'
for i in range(0,6):
    f = open(gameid_vector_path+r'\20210531_'+str(i),'r',encoding='utf-8')
    line = f.readline()
    line_ind = 0
    while line: 
        try:
            gameid,title,vector = line[:-1].split('\t')
            if gameid in gameid_set:
                gameid_vector[gameid] = vector
        except Exception as e:
            # tt = 1
            print(e)
        line = f.readline()
        line_ind += 1
        if line_ind%1000==0:
            print(line_ind)
    f.close()
    

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
too many values to unpack (expected 3)
33000
34000
35000
36000
3700

In [11]:
# 看看训练数据里多少query是在top10000query中的
trainQueryNoHot = set()
trainQueryHot = set()
for itm in train_data_main:
    if itm[0] not in query_vectors.keys():
        trainQueryNoHot.add(itm[0])
    else:
        trainQueryHot.add(itm[0])
        

In [13]:
# 获取这部分noHotQuery的向量

for _ind,query in enumerate(trainQueryNoHot):
    if _ind%400==0:
        print(_ind)
    time.sleep(0.015)
    tmp = getVectorFromEas(query)
    if tmp!='':
        query_vectors[query] = tmp

0
400
800
1200
1600
2000
2400
2800
3200
3600
4000
4400
4800
5200
5600
6000
6400
6800
7200
7600
8000
8400
8800
9200
9600
10000
10400
10800
11200
11600
12000
12400
12800
13200
13600
14000
14400
14800
15200
15600
16000
16400
16800
17200
17600
18000
18400
18800
19200
19600
20000
20400
20800
21200
21600
22000
22400
22800
23200
23600
24000
24400
24800
25200
25600
26000
26400
26800
27200
27600
28000
28400
28800
29200
29600
30000
30400
30800
31200
31600
32000
32400
32800
33200
33600
34000
34400
34800
35200
35600
36000
36400
36800
37200
37600
38000
38400
38800
39200
39600


In [15]:
# 将这所有的query_vectors存到本地
f = open('query_vectors_all','w',encoding='utf-8')
for query,vector in query_vectors.items():
    f.write(query+'\t'+vector+'\n')
f.close()

In [16]:
# 获取游戏的点击转化效能
instance = o.execute_sql("SELECT gameid,ctrcvr FROM metaapp_jarvis.ljq__ctrcvr WHERE ymd = '20210531'")
gameid_ctrcvr = {}
with instance.open_reader() as reader:
    for record in reader:  
        gameid_ctrcvr[record.values[0]] = record.values[1]


In [35]:
# 对每条训练数据，格式化为 query gameid  similarScore qualityScore packageSize(先不用) dayInWeek(先不用) hourInDay(先不用) query_vector
# 将训练数据，query向量，title向量

import numpy as np
train_data = []
for _ind,itm in enumerate(train_data_main):
    try:
        if _ind%1000==0:
            print(_ind)
            # break
        query = itm[0]
        gameid = itm[1]
        playtime = itm[2]
        query_vector = [float(x) for x in query_vectors[query].split(',')]
        similarScore = np.sum(np.array(query_vector)*np.array([float(x) for x in gameid_vector[gameid].split('_')]))
        qualityScore = float(gameid_ctrcvr[gameid])/float(gameid_ctrcvr['-1'])
        train_data.append([query,gameid,playtime,similarScore,qualityScore]+query_vector)
    except Exception as e:
        print(itm)

0
['.植物大战僵尸', '426298', 583767.0]
['233小游戏', '425774', 0.0]
1000
2000
3000
['《山海经》？', '425764', 0.0]
['一分钟', '425783', 0.0]
4000
['三国世界：热血国战无双赵云曹操三国志单机策略手游', '425660', 0.0]
5000
['中巴', '425729', 0.0]
['乐视版植物大战僵尸', '426298', 1860089.0]
6000
7000
8000
9000
['做彩泥', '425694', 1152681.0]
['健身', '425601', 55922.0]
10000
11000
12000
13000
['功夫大师', '425661', 0.0]
['动漫坏女孩', '425698', 2581191.0]
14000
['化妆泥如何使化妆品粘泥', '425694', 332162.0]
['单人小游戏', '425762', 269003.0]
15000
16000
17000
['叠杯子收杯子', '425635', 312418.0]
['史莱姆炫彩软泥模拟器', '425694', 288064.0]
18000
['合成坦克', '425858', 0.0]
['合成枪手', '425858', 52187.0]
19000
20000
['哦，杰克', '425525', 732546.0]
21000
22000
23000
24000
25000
26000
['女神大挑战（我身材最辣）', '425600', 198800.0]
['奶奶', '425597', 1068787.0]
['奶奶冰淇淋：恐怖的游戏国防部', '425597', 253475.0]
27000
28000
29000
['小姐姐海岛求生', '425693', 167856.0]
30000
['小小村长', '425763', 72514.0]
31000
['山海兽', '425764', 28617.0]
['山海经', '425764', 145908.0]
['巡逻警察模拟器', '425528', 84166.0]
32000
33000
['彩泥史莱姆', '425694', 30976.0]

In [36]:
# 将训练数据存到本地
f = open('train_data','w',encoding='utf-8')
for itm in train_data:
    f.write('\t'.join([str(x) for x in itm])+'\n')
f.close()

In [37]:
len(train_data)



90133